Evaluate the models

# Stochastic Adversarial Training (StochAT)

### SoTA

vanila SGD: 
MNIST - 99%+ (most cnns), CIFAR10 - 93%+ (resnet18), 96%+ (wideresnet) 

MNIST:

adversarial attacks: 
l-inf @ eps = 80/255 @20 steps: TRADES - 96.07% - (4 layer cnn), MART 96.4%, MMA 95.5%, PGD - 96.01% - (4 layer cnn)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MMA, PGD

CIFAR10:

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
TRADES 53-56% - (WRN-34-10), MART 57-58% (WRN-34-10), MMA 47%, PGD 48% - (WRN-32-10)// 49% - (WRN-34-10), Std - 0.03%
https://openreview.net/pdf?id=rklOg6EFwS (Table 4)

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
[ResNet10] TRADES 45.4%, MART 46.6%, MMA 37.26%, PGD 42.27%, Std 0.14%

Benign accuracies: TRADES 84.92%, MART 83.62%, MMA 84.36, PGD 87.14%, Std 95.8% [wideresnet]
https://openreview.net/pdf?id=Ms9zjhVB5R (Table 1)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MART, MMA, PGD

TBD: CWinf attacks

## Pretrained models for comparison

download pretrained models and place in ../trainedmodels/MNIST or ../trainedmodels/CIFAR10 respectively

### TRADES :
https://github.com/yaodongyu/TRADES (MNIST: small cnn, CIFAR10: WideResNet34)
### MMA : 
https://github.com/BorealisAI/mma_training (MNIST: lenet5, CIFAR10: WideResNet28)
### MART :
 https://github.com/YisenWang/MART (CIFAR10: ResNet18 and WideResNet34)

## IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms, datasets
from multiprocessing import cpu_count
from collections import OrderedDict
import matplotlib.pyplot as plt
import torch
import olympic
from typing import Union, Callable, Tuple
import sys
sys.path.append('../adversarial/')
sys.path.append('../architectures/')
from functional import boundary, iterated_fgsm, local_search, pgd, entropySmoothing
from ESGD_utils import *
import pickle
import time
import torch.backends.cudnn as cudnn
import argparse, math, random
import ESGD_optim
from trades import trades_loss

In [2]:
from torch.autograd import Variable

In [3]:
from torch.optim.lr_scheduler import StepLR

In [4]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

In [5]:
DEVICE

'cuda'

# LOAD DATA

In [6]:
#place data folders outside working directory

In [7]:
kwargs = {'num_workers': 1, 'pin_memory': True}

In [8]:
args = {}
args['test_batch_size'] = 128
args['no_cuda'] = False
args['epsilon'] = 0.031
args['num_steps'] = 10
args['step_size'] = 0.007
args['random'] =True,
args['white_box_attack']=True
args['log_interval'] = 100
args['beta'] = 6.0
args['seed'] = 1
args['lr'] = 0.1
args['momentum'] = 0.9
args['epochs'] = 5
args['batch_size'] = 128
args['save_freq'] = 5

In [9]:
dataset = 'CIFAR10' # [MNIST, CIFAR10]
transform = transforms.Compose([
    transforms.ToTensor(),
])
bsz = 128
if dataset == 'MNIST':
    train = datasets.MNIST('../../data/MNIST', train=True, transform=transform, download=True)
    val = datasets.MNIST('../../data/MNIST', train=False, transform=transform, download=True)
elif dataset == 'CIFAR10':
    train = datasets.CIFAR10('../../data/CIFAR10', train=True, transform=transform, download=True)
    val = datasets.CIFAR10('../../data/CIFAR10', train=False, transform=transform, download=True)
    
train_loader = DataLoader(train, batch_size=128, shuffle=True, **kwargs)
val_loader = DataLoader(val, batch_size=128, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


# INITIALIZE NETWORK

In [10]:
if dataset=='MNIST':
    from small_cnn import SmallCNN   
    Net = SmallCNN
    NetName = 'SmallCNN'

In [11]:
if dataset=='CIFAR10':
    #[ResNet18,ResNet34,ResNet50,WideResNet]
    from resnet import ResNet18,ResNet34,ResNet50
    from wideresnet import WideResNet
    Net = WideResNet
    NetName = 'WideResNet'

# RANDOM SEED 

In [12]:
seed = 1
torch.set_num_threads(2)
if DEVICE=='cuda':
    torch.cuda.set_device(-1)
    torch.cuda.manual_seed(seed)
    cudnn.benchmark = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# LOAD PRETRAINED OR TRAIN NEW MODELS:

In [13]:
TrainSGD = False
TrainESGD = False
TrainL2 = False
TrainLInf = False
TrainSAT2 = False
TrainSATInf = True
TrainTRADES = False
TrainTRADESInf = True
# tbd: add training modules for MART(inf only), MMA and MMAInf - always keep False
TrainMART = False 
TrainMMA = False
TrainMMAInf = False

# TRAIN NAIVE MODEL USING SGD

In [14]:
def eval_train(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    print('Training: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    training_accuracy = correct / len(train_loader.dataset)
    return train_loss, training_accuracy

In [15]:
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [16]:
def evaluate_against_adversary(model, k, eps, step, norm):
    total = 0
    acc = 0
    for x, y in val_loader:
        total += x.size(0)
        
        if norm == 2:
            x_adv = pgd(
                model, x.to(DEVICE), y.to(DEVICE), torch.nn.CrossEntropyLoss(), k=k, step=step, eps=eps, norm=2)
            #print('rel. l2-norm of x_adv-x:',torch.norm(x_adv.detach().cpu()-x)/np.sqrt(x.size(0)))#/torch.norm(x))
            if torch.norm(x_adv.detach().cpu()-x)/np.sqrt(x.size(0))<0.6*eps :
                print('reconfigure attack')
            elif torch.norm(x_adv.detach().cpu()-x)/np.sqrt(x.size(0))>eps :
                print('reconfigure attack')    
                break;
        elif norm == 'inf':
            x_adv = iterated_fgsm(
                model, x.to(DEVICE), y.to(DEVICE), torch.nn.CrossEntropyLoss(), k=k, step=step, eps=eps, norm='inf')
            #print('rel. linf-norm of x_adv-x:',infnorm(x_adv.detach().cpu()-x)/infnorm(x))
            if infnorm(x_adv.detach().cpu()-x)/infnorm(x)<0.75*eps:
                print('reconfigure attack')
                break;
        y_pred = model(x_adv)

        acc += olympic.metrics.accuracy(y.to(DEVICE), y_pred) * x.size(0)
    print('Adversarial Accuracy: {}/{} ({:.0f}%)'.format(acc,total,100. * acc / total))
    return acc/total

In [17]:
def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=args['epsilon'],
                  num_steps=args['num_steps'],
                  step_size=args['step_size']):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args['random']:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(DEVICE)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    #print('err pgd (white-box): ', err_pgd)
    return err, err_pgd

def eval_adv_test_whitebox(model, device, test_loader):
    """
    evaluate model by white-box attack
    """
    model.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_whitebox(model, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
    print('natural_acc_total: ', 100-natural_err_total.item()/100)
    print('robust_acc_total: ', 100-robust_err_total.item()/100)

In [18]:
if not os.path.exists('../trainedmodels'):
    os.makedirs('../trainedmodels')

In [19]:
def infnorm(x):
    infn = torch.max(torch.abs(x.detach().cpu()))
    return infn

In [20]:
def train_adversarial(method,model, device, train_loader, optimizer, epoch,adversary,k,step,eps,norm,random):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ypred = model(data)
        
        sgd_loss = nn.CrossEntropyLoss()
        # calculate robust loss
        loss = method(model,optimizer,sgd_loss,data,target,epoch,adversary,k,step,eps,norm,random)

# TRAIN MODEL USING SAT

In [21]:
def adversarial_training_entropy(model, optimiser, loss_fn, x, y, epoch, adversary, k, step, eps, norm, random):
    """Performs a single update against a specified adversary"""
    model.train()
    
    # Adversial perturbation
    N = k
    alpha=0.8
    loss = 0
    advs = []
    for l in range(N):
        
        if l==0:
            k=1
            random=True
            xp = None
            projector=False
        elif l>0 and l<N-1:
            k=1
            random=False
            xp=x_adv
            projector = False
        elif l == N-1:
            k=1
            random=False
            xp = x_adv
            projector=True
        x_adv = adversary(model, x, y, loss_fn, xp=xp, k=k, step=step, step2=0.05, eps=eps, norm=norm, random=random, gamma=0.3, ep=3e-3,projector=projector, debug=False)
        #x_adv = adversary(model, x, y, loss_fn, k=k, step=step, eps=eps, norm=norm, random=True)
        #print('l2:',torch.norm(x_adv.detach().cpu()-x.detach().cpu())/np.sqrt(x.detach().cpu().size(0)))    
        #diff = x_adv.view(-1)-x.view(-1)
        #advs.append(diff)
        #print('linf:',infnorm(x_adv.detach().cpu()-x.detach().cpu())/infnorm(x)) 
        #if l>0:
        #    print('difference angle:',torch.acos(torch.dot(diff,advs[0]/(torch.norm(diff)*torch.norm(advs[-2])))))
        optimiser.zero_grad()
        y_pred = model(x_adv)
        #loss = loss + loss_fn(y_pred,y)
        loss = (1-alpha)*loss + alpha*loss_fn(y_pred, y)
    #loss = loss/N
    loss.backward()
    optimiser.step()
    return loss, y_pred

In [22]:
model_dir = '../WRN_ATENT'

In [23]:
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [24]:
def adjust_learning_rate(optimizer, epoch,lr_init):
    """decrease the learning rate"""
    lr = lr_init
    if epoch >= 75:
        lr = lr_init * 0.1
    if epoch >= 90:
        lr = lr_init * 0.01
    if epoch >= 100:
        lr = lr_init * 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [25]:
## initialize model
model_SATInf = Net().to(DEVICE)

## training params
epochs = 100
lr_init = 0.15 #0.03-96.65
optimizer = optim.SGD(model_SATInf.parameters(), lr=lr_init, momentum=0.9)
scheduler = StepLR(optimizer, step_size=40, gamma=0.1)    
## train model

for epoch in range(1, epochs + 1):
    print('Epoch:',epoch)
    # adjust learning rate for SGD
    adjust_learning_rate(optimizer, epoch,lr_init)
    print('LR:')
    for param_group in optimizer.param_groups:
        print(param_group['lr'])

    # adversarial training
    train_adversarial(adversarial_training_entropy,model_SATInf, DEVICE, train_loader, optimizer, epoch,adversary=entropySmoothing,k=10,step=0.007,eps=0.031,norm='inf',random=False)
    #train_adversarial(adversarial_training,model_SATInf, DEVICE, train_loader, optimizer, epoch,adversary=iterated_fgsm,k=40,step=0.01,eps=0.3,norm='inf',random=False)

    # evaluation on natural examples

    if (epoch) % 5 == 0:
        print('================================================================')
        #eval_train(model_SATInf, DEVICE, train_loader)
        #eval_test(model_SATInf, DEVICE, val_loader)
        eval_adv_test_whitebox(model_SATInf, DEVICE, val_loader)            
        print('================================================================')
    #scheduler.step()


     # save checkpoint
    if epoch % args['save_freq'] == 0:
        torch.save(model_SATInf.state_dict(),
                   os.path.join(model_dir, 'model-nn-epoch{}.pt'.format(epoch)))
        torch.save(optimizer.state_dict(),
                   os.path.join(model_dir, 'opt-nn-checkpoint_epoch{}.tar'.format(epoch)))
## save model
modelname = '../trainedmodels/'+dataset+'/'+NetName+'_SATInf_ep'+str(epochs)+'_lr'+str(lr_init)+'.pt'
torch.save(model_SATInf,modelname)

Epoch: 1
LR:
0.15


RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 23.65 GiB total capacity; 22.25 GiB already allocated; 60.56 MiB free; 22.61 GiB reserved in total by PyTorch)